In [2]:
import scipy.sparse as sp
import numpy as np

In [73]:
def delete_row_lil(mat, i):
    if not isinstance(mat, sp.lil_matrix):
        raise ValueError("works only for LIL format -- use .tolil() first")
    mat.rows = np.delete(mat.rows, i)
    mat.data = np.delete(mat.data, i)
    mat._shape = (mat._shape[0] - 1, mat._shape[1])

def add_row_lil(mat,r):
    return spsp.vstack((mat,r))

In [38]:
def MHS( conflist ):
    pass
def SPMHS( tr, d, n):
    pass

In [183]:
conflist = [np.array([0,1,2],dtype=np.int64),np.array([0,1,3],dtype=np.int64)]
ncomps = np.max(map(lambda c: np.max(c), conflist))+1
nconf = len(conflist)
confmat = np.zeros((nconf,ncomps),dtype=np.int64)
for k,c in enumerate(conflist):
    confmat[k,c]=1
trs = sp.csr_matrix(confmat)
mmhs = sp.lil_matrix((1,ncomps),dtype=np.int64)

In [184]:
tr = trs[0,:]
dnew=sp.lil_matrix((0,ncomps), dtype=np.int64)
k=0
while mmhs.shape[0]>=k:
    print "p10: " + str(k)
    print "p20: " + str(mmhs.shape)
    if not np.any(tr.toarray()*mmhs[k,:].toarray()): # No intersection between tr and mmhs[k,:]
        tmp = mmhs[k,:].copy()
        idx = tr.indices
        delete_row_lil(mmhs,k)
        k = k-1
        for l in idx:
            cand = tmp.copy()
            cand[0,l] = 1
            candmin = True
            j=0
            print "p30: " + str(mmhs.shape)
            print "p31: " + str(cand.todense())
            while candmin and j < mmhs.shape[0]:
                print "p33: " + str(type(mmhs))
                print "p35: " + str(mmhs[j,:].toarray())
                if np.all(mmhs[j,:].toarray()*cand.toarray()==mmhs[j,:].toarray()):
                    candmin = False
                    break
                j = j+1
            print "p40: " + str(candmin)
            if candmin:
                print "p42: "  + str(type(mmhs))
                mmhs = add_row_lil(mmhs,cand)
                print "p44: "  + str(type(mmhs))

    k = k+1

p10: 0
p20: (1, 4)
p30: (0, 4)
p31: [[1 0 0 0]]
p40: True
p42: <class 'scipy.sparse.lil.lil_matrix'>
p44: <class 'scipy.sparse.coo.coo_matrix'>
p30: (1, 4)
p31: [[0 1 0 0]]
p33: <class 'scipy.sparse.coo.coo_matrix'>


TypeError: 'coo_matrix' object has no attribute '__getitem__'

In [186]:
a = sp.lil_matrix(np.eye(3),dtype=np.int64)
print a.todense()

[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [68]:
def MHS(conflist):
    mmhs = [np.array([],dtype=np.int64)]
    for c in conflist:
        j=0
        while j < len(mmhs):
            if len(np.intersect1d(c,mmhs[j]))==0:
                tmp = mmhs.pop(j)
                j = j-1
                for ci in c:
                    newCand = np.concatenate((tmp,[ci]))
                    candmin = True
                    k=0
                    while candmin and k<len(mmhs):
                        if len([x for x in mmhs[k] if x in newCand])==len(mmhs[k]):
                            candmin = False
                            break
                        k=k+1
                    if candmin:
                        mmhs.append(newCand)
            j = j+1
        return np.array(mmhs)
    
conflist = [np.array([0,1,2],dtype=np.int64),np.array([0,1,3],dtype=np.int64)]
MHS(conflist)

array([[0],
       [1],
       [2]])